In [ ]:
import pandas as pd

from gettsim import main
from ttsim import InputData, MainTarget, TTTargets
from ttsim.tt_dag_elements import ScalarParam

# Prototypes of GETTSIM's new interface

[GEP 7](https://gettsim--855.org.readthedocs.build/en/855/geps/gep-07.html) discusses
the principles of the new interface. This notebook demonstrates two candidates for
GETTIM's new interface. We would like to get your feedback on which one you prefer.

In this notebook, we compute income taxes and social security contributions for example
data.

## Setup

This notebook requires to have GETTSIM installed in its current development version.

To do this:
1. Clone the GETTSIM repository.
2. Install the [pixi package manager](https://pixi.sh/latest/) on your system.
3. Open your shell, `cd` into the GETTSIM repository and run `git checkout collect-components-of-namespaces`.
4. Run `pixi run jupyter-notebook` and select the `interface-playground.ipynb` notebook.

If you have trouble with the setup, please reach out.

## Creating the Data

The first step in GETTSIM's new workflow is to define the targets you're interested in.
The keys of the nested dictionary below are the paths GETTSIM will use as targets. For
instance, via the keys `einkommensteuer` and `betrag_m_sn`, we request the income tax as
a target.

The values on the lowest level of the dictionaries will be used as the column names of
the resulting DataFrame. Here, `income_tax_y` will be the name of the column containing
the income tax results.

In this example, we are interested in the income tax and the social security contributions
paid when being in regular employment.

In [ ]:
TT_TARGETS = {
    "einkommensteuer": {"betrag_y_sn": "income_tax_y"},
    "sozialversicherung": {
        "pflege": {
            "beitrag": {
                "betrag_versicherter_m": "long_term_care_insurance_contribution_m"
            }
        },
        "kranken": {
            "beitrag": {"betrag_versicherter_m": "health_insurance_contribution_m"}
        },
        "rente": {
            "beitrag": {"betrag_versicherter_m": "pension_insurance_contribution_m"}
        },
        "arbeitslosen": {
            "beitrag": {
                "betrag_versicherter_m": "unemployment_insurance_contribution_m"
            }
        },
    },
}

Next, we need to find out which input data we actually need to calculate the targets we
are interested in we can do this by specifying a template as `main_target` of `main`.

Because we are interested social security contributions paid when being in regular
employment, we are not interested in retirees or households dependent on social
assistance. We can override these transfers when requesting the template. This removes
the input data needed to compute these transfers from the template.

In [ ]:
main(
    date_str="2025-01-01",
    main_target=MainTarget.templates.input_data_dtypes,
    tt_targets=TTTargets(
        tree=TT_TARGETS,
    ),
    input_data=InputData.tree(
        {
            "p_id": [0],
            "sozialversicherung": {
                "rente": {
                    "altersrente": {"betrag_m": [0.0]},
                },
                "arbeitslosen": {"betrag_m": [0.0]},
            },
            "wohngeld": {"betrag_m_wthh": [0.0]},
            "kinderzuschlag": {"betrag_m_bg": [0.0]},
            "elterngeld": {"betrag_m": [0.0]},
            "arbeitslosengeld_2": {"betrag_m_bg": [0.0]},
        }
    ),
)

Now, we create some example data. Here, we use a pandas DataFrame with column names that are different from the ones GETTSIM expects.

In [ ]:
DATA = pd.DataFrame(
    {
        "age": [30, 30, 10],
        "working_hours": [35, 35, 0],
        "disability_grade": [0, 0, 0],
        "birth_year": [1995, 1995, 2015],
        "hh_id": [0, 0, 0],
        "p_id": [0, 1, 2],
        "east_germany": [False, False, False],
        "self_employed": [False, False, False],
        "income_from_self_employment": [0, 0, 0],
        "income_from_rent": [0, 0, 0],
        "income_from_employment": [5000, 4000, 0],
        "income_from_forest_and_agriculture": [0, 0, 0],
        "income_from_capital": [500, 0, 0],
        "income_from_other_sources": [0, 0, 0],
        "pension_income": [0, 0, 0],
        "contribution_to_private_pension_insurance": [0, 0, 0],
        "childcare_expenses": [0, 0, 0],
        "person_that_pays_childcare_expenses": [-1, -1, 0],
        "joint_taxation": [True, True, False],
        "amount_private_pension_income": [0, 0, 0],
        "contribution_private_health_insurance": [0, 0, 0],
        "has_children": [True, True, False],
        "single_parent": [False, False, False],
        "is_child": [False, False, True],
        "spouse_id": [1, 0, -1],
        "parent_id_1": [-1, -1, 0],
        "parent_id_2": [-1, -1, 1],
        "in_training": [False, False, False],
        "id_recipient_child_allowance": [-1, -1, 0],
    }
)

Next, we define a mapping from GETTSIM's expected input structure to your data. The idea
is that you save the template we created above to your disk (e.g. as a yaml file) and
edit it there. Then you can read in the file and use its content as the mapper.

For convenience, we directly define the mapper below.

In [ ]:
MAPPER = {
    "alter": "age",
    "arbeitsstunden_w": "working_hours",
    "behinderungsgrad": "disability_grade",
    "geburtsjahr": "birth_year",
    "hh_id": "hh_id",
    "p_id": "p_id",
    "wohnort_ost": "east_germany",
    "einkommensteuer": {
        "einkünfte": {
            "ist_selbstständig": "self_employed",
            "aus_gewerbebetrieb": {"betrag_m": "income_from_self_employment"},
            "aus_vermietung_und_verpachtung": {"betrag_m": "income_from_rent"},
            "aus_nichtselbstständiger_arbeit": {
                "bruttolohn_m": "income_from_employment"
            },
            "aus_forst_und_landwirtschaft": {
                "betrag_m": "income_from_forest_and_agriculture"
            },
            "aus_selbstständiger_arbeit": {"betrag_m": "income_from_self_employment"},
            "aus_kapitalvermögen": {"kapitalerträge_m": "income_from_capital"},
            "sonstige": {
                "ohne_renten_m": "income_from_other_sources",
                "renteneinkünfte_m": "pension_income",
            },
        },
        "abzüge": {
            "beitrag_private_rentenversicherung_m": "contribution_to_private_pension_insurance",  # noqa: E501
            "kinderbetreuungskosten_m": "childcare_expenses",
            "p_id_kinderbetreuungskostenträger": "person_that_pays_childcare_expenses",
        },
        "gemeinsam_veranlagt": "joint_taxation",
    },
    "sozialversicherung": {
        "arbeitslosen": {"betrag_m": 0.0},
        "rente": {
            "private_rente_betrag_m": "amount_private_pension_income",
            "altersrente": {
                "betrag_m": 0.0,
            },
        },
        "kranken": {
            "beitrag": {"privat_versichert": "contribution_private_health_insurance"}
        },
        "pflege": {"beitrag": {"hat_kinder": "has_children"}},
    },
    "familie": {
        "alleinerziehend": "single_parent",
        "kind": "is_child",
        "p_id_ehepartner": "spouse_id",
        "p_id_elternteil_1": "parent_id_1",
        "p_id_elternteil_2": "parent_id_2",
    },
    "wohngeld": {
        "betrag_m_wthh": 0.0,
    },
    "kinderzuschlag": {
        "betrag_m_bg": 0.0,
    },
    "elterngeld": {
        "betrag_m": 0.0,
    },
    "arbeitslosengeld_2": {
        "betrag_m_bg": 0.0,
    },
    "kindergeld": {
        "in_ausbildung": "in_training",
        "p_id_empfänger": "id_recipient_child_allowance",
    },
}

Note: When writing and reading the template to your disk, don't forget to allow for
unicode characters. This is important because many transfers have Umlaute in their
names. An example could look like this:

```python
import yaml

# Write the template to your disk...
with PATH_FOR_TEMPLATE.open("w") as f:
    yaml.dump(TEMPLATE, f, allow_unicode=True)

# Edit the leafs in the template and then read it back in
with PATH_FOR_TEMPLATE.open("r") as f:
    MAPPER = {yaml.load(f, allow_unicode=True)}
```

## Using GETTSIM's interface

Just as for taxes and transfers, GETTSIM's infrastructure is a DAG. GETTSIM's interface
is a function that allows you to interact with this DAG. This comes with the
advantages GETTSIM's users already know from the taxes and transfers part:
- Users can select any part of the DAG as a target. This means that users can access
  any intermediate objects.
- Users can feed any part of the DAG as input. This means that users can overwrite
  specific parts of the DAG (e.g. the policy environment).
- Users can decide which parts of the DAG not to compute. For example, users can choose
  not to perform safety checks on the input data. This means that GETTSIM is quicker in
  computing the result (at the expense of informative errors).

First, we look at the one-stop shop: computing the targets defined above using the input
data. In a second example, we manipulate the policy environment to see why the interface
DAG is useful.

### One-stop-shop: Computing taxes and transfers with GETTSIM

Let's calculate taxes and transfers first:

In [ ]:
result = main(
    date_str="2025-01-01",
    input_data=InputData.df_and_mapper(
        df=DATA,
        mapper=MAPPER,
    ),
    main_target=MainTarget.results.df_with_mapper,
    tt_targets=TTTargets(
        tree=TT_TARGETS,
    ),
)
result.T

### Manipulating the policy environment

First, we obtain the policy environment for the policy date we're interested in. Similar
to above, we have to call the `main` function.

In [ ]:
original_policy_environment = main(
    date_str="2025-01-01",
    main_target=MainTarget.policy_environment,
)

Now, we slightly modify the policy environment. In this simple example, we increase the
contribution rate of the public pension insurance by 1 percentage point.

The contribution rate is a `ScalarParam` object:

In [ ]:
type(
    original_policy_environment["sozialversicherung"]["rente"]["beitrag"][
        "beitragssatz"
    ]
)

We replace the `ScalarParam` object with a new one. Then, we add this parameter to the
new policy environment.

In [ ]:
old_beitragssatz = original_policy_environment["sozialversicherung"]["rente"][
    "beitrag"
]["beitragssatz"]
new_beitragssatz = ScalarParam(value=old_beitragssatz.value + 0.01)

modified_policy_environment = original_policy_environment.copy()
modified_policy_environment["sozialversicherung"]["rente"]["beitrag"][
    "beitragssatz"
] = new_beitragssatz

Now we can compute taxes and transfers using the modified policy environment.

In [ ]:
result = main(
    date_str="2025-01-01",
    policy_environment=modified_policy_environment,
    input_data=InputData.df_and_mapper(
        df=DATA,
        mapper=MAPPER,
    ),
    main_target=MainTarget.results.df_with_mapper,
    tt_targets=TTTargets(
        tree=TT_TARGETS,
    ),
)
result.T

In [ ]:
tt_function = main(
    date_str="2025-01-01",
    input_data=InputData.df_and_mapper(
        df=DATA,
        mapper=MAPPER,
    ),
    main_target=MainTarget.specialized_environment.tax_transfer_function,
    tt_targets=TTTargets(
        tree=TT_TARGETS,
    ),
)